# Embeddings Neuronales


**TODO**

### Importanción de librería requeridas

In [1]:
import gensim.corpora as corpora
from gensim.models import Word2Vec

from sklearn.cluster import KMeans
import numpy as np

import pickle
import pandas as pd

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline


### Definición de variables globales

In [2]:
TEXT_FILE_READ = 'docs/preprocessing_reddit_data.csv'
TEXT_SAVE_FILE = 'docs/reddit_data_lda.csv'
FILENAME_PICKLE = "docs/tmpreddit.pickle"
n_clusters = 70

### Lectura de los comentarios de Reddit

Los comentarios fueron previamente preprocesados (Ver en TODO).

In [3]:
with open(FILENAME_PICKLE, 'rb') as f:
    df = pickle.load(f)


### Vocabulario

In [4]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemma_tokens'])

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)

# Creating a corpus object
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]


In [5]:
processed_corpus = df['lemma_tokens']


### Entrenamiento del modelo Word2Vec

In [6]:
model = Word2Vec(sentences=processed_corpus, vector_size=100, window=5, min_count=1, workers=4)
model.train(processed_corpus, total_examples=len(processed_corpus), epochs=100)
model.save("word2vec.model")

In [7]:
word_vecs = []
vocabulary = list(model.wv.key_to_index)

for key in model.wv.key_to_index:
    word_vecs.append(model.wv[key])

In [8]:
# algunas predicciones

model.wv.most_similar("rucula")

[('jajajajjajaajaj', 0.9570168256759644),
 ('bottle', 0.9228257536888123),
 ('laconcho', 0.9012131094932556),
 ('branding', 0.8885334134101868),
 ('submarinar', 0.8106595873832703),
 ('meanies', 0.7711156010627747),
 ('golden', 0.7673962116241455),
 ('frondizi', 0.6835584044456482),
 ('gustavo', 0.676913321018219),
 ('oct', 0.6762269139289856)]

In [9]:
# generamos los clústers

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
X_wvkm = kmeans.fit_transform(word_vecs)
y_wvkm = kmeans.predict(word_vecs)

In [12]:
for cluster in range(n_clusters):
    mask = X_wvkm[y_wvkm == cluster]
    idx_sort = np.argsort(X_wvkm[:,cluster])
    words = [vocabulary[x] for x in idx_sort[:20]]

    print("Clúster %d:" % cluster, end='')
    print()
    for w in words:
        print(' %s' % w, end='')
    print()

In [17]:
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

In [21]:
X = model.transform(["Nothing is easy in cricket. Maybe when you watch it on TV, it looks easy. But it is not. You have to use your brain and time the ball."])
predicted = model.predict(X)
print(predicted)

AttributeError: 'Word2Vec' object has no attribute 'transform'

In [14]:
#reddit = pd.read_csv(TEXT_FILE_READ)

test = kmeans.predict(['peron k'])

ValueError: could not convert string to float: 'peron k'

In [15]:
reddit = pd.read_csv(TEXT_FILE_READ)

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # row = sorted(row, key=lambda x: (x[1]), reverse=True) # old line
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                #ent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4)]), ignore_index=True)
                #print(sent_topics_df)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    #sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution']

    # Add original text to the end of the output
    #contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, texts], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=base_model, corpus=corpus, texts=reddit)

NameError: name 'base_model' is not defined

In [10]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
#df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,index,Dominant_Topic,Perc_Contribution,Topic_Keywords,score,id,flair,comms_num,body,comment_parent_id,is_replay,lemma_tokens,body_preprocessing
0,0,12.0,0.2531,"él, recordar, pegar, único, robar, barrio, pen...",1,hgw77qe,Política🏛️,0,"Iba a decir, bue si lo saco de su bolsillo... ...",q9imco,False,"['bue', 'saco', 'bolsillo', 'recorder', 'hdp',...",bue saco bolsillo recorder hdp mantener alcanz...
1,1,2.0,0.2598,"perro, nik, meme, gobierno, explicar, it, teni...",1,hgw7dci,Política🏛️,0,Se volvio un meme el bot del dolar?,hgw666m,True,"['volvio', 'meme', 'dolar']",volvio meme dolar
2,2,26.0,0.3279,"falacia, decir, gratis, k, joda, país, mandar,...",1,hgw69er,Humor:snoo_joy:,0,Este Esteban Lamothe estaba en la ficción de u...,q9i4uj,False,"['ester', 'lamothe', 'ficción', 'villo', 'acá'...",ester lamothe ficción villo acá comedia políti...
3,3,6.0,0.4349,"pobre, servir, él, comida, tenés, culpa, onda,...",1,hgw6zvd,Meme💩,0,Eso porque son todos útos chupa bija.. Venga e...,hgw2528,True,"['úto', 'chupa', 'bijo', 'venir', 'ban', 'nedf...",úto chupa bijo venir ban nedflanducacion
4,4,21.0,0.8089,"re, cabeza, él, morir, pibes, papa, hambre, ri...",1,hgw24ns,Meme💩,0,mas verso burgués que Maximo no hay. Es la rep...,q9hut7,False,"['verso', 'burgués', 'maximo', 'representación']",verso burgués maximo representación
5,5,12.0,0.3722,"él, recordar, pegar, único, robar, barrio, pen...",1,hgw38x8,Meme💩,0,Ayudar con comida? Na mejor unos afiches a tod...,q9hut7,False,"['ayudar', 'comida', 'na', 'afich', 'color']",ayudar comida na afich color
6,6,17.0,0.3461,"the, of, necesitar, you, fácil, and, to, creer...",1,hgw2rml,Meme💩,1,¿Por qué si es un cerdo tiene 6 patas?,q9hut7,False,"['cerdo', 'pata']",cerdo pata
7,7,22.0,0.3515,"ah, /s, peronista, paso, x200b, mes, cagar, él...",1,hgw3wei,Meme💩,0,"Mira, soy tan capitalista que por 15 mil pesos...",q9hut7,False,"['mira', 'capitalisto', 'pesos', 'corrijo', 'c...",mira capitalisto pesos corrijo color
8,8,24.0,0.4082,"milei, pasar, debate, votar, voto, mujer, izqu...",1,hgw78bv,Meme💩,0,Swinetaur libertario de Darkest Perónia. Ruin ...,q9hut7,False,"['swinetaur', 'libertario', 'darkest', 'peróni...",swinetaur libertario darkest perónia ruin come...
9,9,23.0,0.4081,"él, foto, ver, libertad, sacar, feriado, tomar...",1,hgw6rim,Meme💩,0,como no pueden contra elllll. lo ensucian vamo...,q9hut7,False,"['elllll', 'ensuciar', 'milie', 'bastar', 'k']",elllll ensuciar milie bastar k


In [11]:
df_dominant_topic.to_csv(TEXT_SAVE_FILE, index=False)